# Quantitative Value Strategy
Building an investing strategy that selects the 50 stocks with the best value metrics. 
This algorithm will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing 
List of Stocks & API Token

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from donotshow import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN

'sk_fff2432d44774393b45c89a5883316c5'

## API Call
Ranking securities based on a single metric (price-to-earnings ratio).

In [3]:
symbol = 'aapl'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
data = data.json()
data = data[0]
data

{'avgTotalVolume': 58657720,
 'calculationPrice': 'tops',
 'change': 2.97,
 'changePercent': 0.01607,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 187.78,
 'iexAskSize': 100,
 'iexBidPrice': 187.77,
 'iexBidSize': 100,
 'iexClose': 187.77,
 'iexCloseTime': 1699994096007,
 'iexLastUpdated': 1699994096007,
 'iexMarketPercent': 0.027103756834636568,
 'iexOpen': 187.69,
 'iexOpenTime': 1699972200222,
 'iexRealtimePrice': 187.77,
 'iexRealtimeSize': 100,
 'iexVolume': 1170778,
 'lastTradeTime': 1699994096007,
 'latestPrice': 187.77,
 'latestSource': 'IEX real time price',
 'latestTime': '3:34:56 PM',
 'latestUpdate': 1699994096007,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime'

## Parsing API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

30.63

## Batch API Call

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

## Dataframe of Quote Data

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
temp_list = []

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for i, symbol in enumerate(symbol_string.split(',')):
        temp_list.append(pd.Series([symbol, data[i]['latestPrice'], 
                                    data[i]['peRatio'],'N/A'], index = my_columns))

In [7]:
final_dataframe = pd.DataFrame(temp_list, columns = my_columns)
temp_list

[Ticker                         A
 Price                      112.1
 Price-to-Earnings Ratio    29.35
 Number of Shares to Buy      N/A
 dtype: object,
 Ticker                        AAL
 Price                      12.275
 Price-to-Earnings Ratio      4.58
 Number of Shares to Buy       N/A
 dtype: object,
 Ticker                       AAP
 Price                      58.79
 Price-to-Earnings Ratio    10.19
 Number of Shares to Buy      N/A
 dtype: object,
 Ticker                       AAPL
 Price                      187.77
 Price-to-Earnings Ratio     30.63
 Number of Shares to Buy       N/A
 dtype: object,
 Ticker                       ABBV
 Price                      138.25
 Price-to-Earnings Ratio     37.88
 Number of Shares to Buy       N/A
 dtype: object,
 Ticker                        ABC
 Price                      179.98
 Price-to-Earnings Ratio     21.98
 Number of Shares to Buy       N/A
 dtype: object,
 Ticker                       ABMD
 Price                      381.02
 P

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, the next step is to remove glamour stocks from the DataFrame.

Sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy

Buying an equivalent value of all 50 stocks identified by their P/E ratio

In [9]:
portfolio_size = 500000

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BRK.B,355.110,0.01,28
1,FRC,3.510,0.43,2849
2,DISH,3.460,1.74,2890
3,ZION,35.500,3.79,281
4,EMR,89.025,3.92,112
5,GM,28.290,3.98,353
6,VLO,126.330,4.30,79
7,ALB,125.470,4.45,79
8,AAL,12.275,4.58,814
9,UAL,39.450,4.58,253


## Combined value metrics used for Scoring
 Filtering for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data.

In [11]:
symbol = 'AAPL'
advanced_stats = 0
quote = 1
batch_api_call_url = f'https://api.iex.cloud/v1/data/core/ADVANCED_STATS,QUOTE/{symbol}?token={IEX_CLOUD_API_TOKEN}'
                     #f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[quote]['peRatio']

# P/B Ratio
pb_ratio = data[advanced_stats]['priceToBook']

#P/S Ratio
ps_ratio = data[advanced_stats]['priceToSales']

# EV/EBITDA
enterprise_value = data[advanced_stats]['enterpriseValue']
ebitda = data[advanced_stats]['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[advanced_stats]['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

cv = combined value

In [12]:
cv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'CV Score'
]

temp_list = []


for symbol_string in symbol_strings:
    advanced_stats_data_url = f'https://api.iex.cloud/v1/data/core/ADVANCED_STATS/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    quote_data_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    advanced_stats_data = requests.get(advanced_stats_data_url).json()
    quote_data = requests.get(quote_data_url).json()
    for i, symbol in enumerate(symbol_string.split(',')):
        enterprise_value = advanced_stats_data[i]['enterpriseValue']
        ebitda = advanced_stats_data[i]['EBITDA']
        gross_profit = advanced_stats_data[i]['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        temp_list.append(
            pd.Series([
                symbol,
                quote_data[i]['latestPrice'],
                'N/A',
                quote_data[i]['peRatio'],
                'N/A',
                advanced_stats_data[i]['priceToBook'],
                'N/A',
                advanced_stats_data[i]['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ], index = cv_columns)),

In [13]:
cv_dataframe = pd.DataFrame(temp_list, columns = cv_columns)
cv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,CV Score
0,A,112.100,N/A,29.35,N/A,5.660,N/A,4.5000,N/A,19.410872,N/A,9.236785,N/A,N/A
1,AAL,12.275,N/A,4.58,N/A,-1.500,N/A,0.1455,N/A,4.586499,N/A,0.578405,N/A,N/A
2,AAP,58.790,N/A,10.19,N/A,1.220,N/A,0.2965,N/A,5.927713,N/A,1.008424,N/A,N/A
3,AAPL,187.785,N/A,30.63,N/A,46.250,N/A,7.5000,N/A,23.237018,N/A,17.284754,N/A,N/A
4,ABBV,138.250,N/A,37.88,N/A,20.240,N/A,4.4400,N/A,11.623647,N/A,8.059964,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.980,N/A,24.33,N/A,-4.290,N/A,4.9800,N/A,19.148386,N/A,8.688029,N/A,N/A
501,ZBH,108.300,N/A,47.92,N/A,1.770,N/A,3.0400,N/A,14.243421,N/A,5.236693,N/A,N/A
502,ZBRA,216.170,N/A,24.10,N/A,3.450,N/A,2.0500,N/A,14.470148,N/A,5.346597,N/A,N/A
503,ZION,35.500,N/A,3.79,N/A,0.994,N/A,0.9717,N/A,2.668473,N/A,1.007055,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
cv_dataframe[cv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,CV Score
6,ABMD,381.020,N/A,65.69,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,182.500,N/A,59.45,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,334.550,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
48,ATVI,94.420,N/A,34.59,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,94.920,N/A,47.94,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.73,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,103.900,N/A,39.96,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,24.420,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.260,N/A,14.27,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
184,FISV,114.230,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    cv_dataframe[column].fillna(cv_dataframe[column].mean(), inplace = True)

In [16]:
cv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,CV Score
0,A,112.100,N/A,29.35,N/A,5.660,N/A,4.5000,N/A,19.410872,N/A,9.236785,N/A,N/A
1,AAL,12.275,N/A,4.58,N/A,-1.500,N/A,0.1455,N/A,4.586499,N/A,0.578405,N/A,N/A
2,AAP,58.790,N/A,10.19,N/A,1.220,N/A,0.2965,N/A,5.927713,N/A,1.008424,N/A,N/A
3,AAPL,187.785,N/A,30.63,N/A,46.250,N/A,7.5000,N/A,23.237018,N/A,17.284754,N/A,N/A
4,ABBV,138.250,N/A,37.88,N/A,20.240,N/A,4.4400,N/A,11.623647,N/A,8.059964,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.980,N/A,24.33,N/A,-4.290,N/A,4.9800,N/A,19.148386,N/A,8.688029,N/A,N/A
501,ZBH,108.300,N/A,47.92,N/A,1.770,N/A,3.0400,N/A,14.243421,N/A,5.236693,N/A,N/A
502,ZBRA,216.170,N/A,24.10,N/A,3.450,N/A,2.0500,N/A,14.470148,N/A,5.346597,N/A,N/A
503,ZION,35.500,N/A,3.79,N/A,0.994,N/A,0.9717,N/A,2.668473,N/A,1.007055,N/A,N/A


## Calculating Value Percentiles

Calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in cv_dataframe.index:
    for metric in metrics.keys():
        cv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(cv_dataframe[metric], cv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(cv_dataframe[metric])

#Print the entire DataFrame    
cv_dataframe

0       0.70099
1       0.09604
2      0.194059
3      0.724752
4      0.835644
         ...   
500         0.6
501     0.89505
502    0.592079
503    0.085149
504     0.80198
Name: PE Percentile, Length: 505, dtype: object
0      0.770297
1       0.10297
2      0.220792
3      0.976238
4      0.958416
         ...   
500     0.09901
501    0.365347
502    0.613861
503    0.180198
504    0.928713
Name: PB Percentile, Length: 505, dtype: object
0      0.776238
1      0.013861
2      0.039604
3       0.90297
4      0.768317
         ...   
500     0.80198
501     0.59901
502    0.461386
503    0.185149
504    0.936634
Name: PS Percentile, Length: 505, dtype: object
0           0.8
1      0.059406
2      0.116832
3      0.877228
4      0.409901
         ...   
500     0.79604
501    0.538614
502    0.546535
503    0.019802
504    0.910891
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.746535
1      0.019802
2      0.063366
3      0.958416
4      0.669307
         ...   
5

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,CV Score
0,A,112.100,N/A,29.35,0.70099,5.660,0.770297,4.5000,0.776238,19.410872,0.8,9.236785,0.746535,N/A
1,AAL,12.275,N/A,4.58,0.09604,-1.500,0.10297,0.1455,0.013861,4.586499,0.059406,0.578405,0.019802,N/A
2,AAP,58.790,N/A,10.19,0.194059,1.220,0.220792,0.2965,0.039604,5.927713,0.116832,1.008424,0.063366,N/A
3,AAPL,187.785,N/A,30.63,0.724752,46.250,0.976238,7.5000,0.90297,23.237018,0.877228,17.284754,0.958416,N/A
4,ABBV,138.250,N/A,37.88,0.835644,20.240,0.958416,4.4400,0.768317,11.623647,0.409901,8.059964,0.669307,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.980,N/A,24.33,0.6,-4.290,0.09901,4.9800,0.80198,19.148386,0.79604,8.688029,0.714851,N/A
501,ZBH,108.300,N/A,47.92,0.89505,1.770,0.365347,3.0400,0.59901,14.243421,0.538614,5.236693,0.39802,N/A
502,ZBRA,216.170,N/A,24.10,0.592079,3.450,0.613861,2.0500,0.461386,14.470148,0.546535,5.346597,0.407921,N/A
503,ZION,35.500,N/A,3.79,0.085149,0.994,0.180198,0.9717,0.185149,2.668473,0.019802,1.007055,0.061386,N/A


## Calculating the CV Score
The CV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in cv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(cv_dataframe.loc[row, metrics[metric]])
    cv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
cv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,CV Score,RV Score
0,A,112.100,N/A,29.35,0.70099,5.660,0.770297,4.5000,0.776238,19.410872,0.8,9.236785,0.746535,N/A,0.758812
1,AAL,12.275,N/A,4.58,0.09604,-1.500,0.10297,0.1455,0.013861,4.586499,0.059406,0.578405,0.019802,N/A,0.058416
2,AAP,58.790,N/A,10.19,0.194059,1.220,0.220792,0.2965,0.039604,5.927713,0.116832,1.008424,0.063366,N/A,0.126931
3,AAPL,187.785,N/A,30.63,0.724752,46.250,0.976238,7.5000,0.90297,23.237018,0.877228,17.284754,0.958416,N/A,0.887921
4,ABBV,138.250,N/A,37.88,0.835644,20.240,0.958416,4.4400,0.768317,11.623647,0.409901,8.059964,0.669307,N/A,0.728317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.980,N/A,24.33,0.6,-4.290,0.09901,4.9800,0.80198,19.148386,0.79604,8.688029,0.714851,N/A,0.602376
501,ZBH,108.300,N/A,47.92,0.89505,1.770,0.365347,3.0400,0.59901,14.243421,0.538614,5.236693,0.39802,N/A,0.559208
502,ZBRA,216.170,N/A,24.10,0.592079,3.450,0.613861,2.0500,0.461386,14.470148,0.546535,5.346597,0.407921,N/A,0.524356
503,ZION,35.500,N/A,3.79,0.085149,0.994,0.180198,0.9717,0.185149,2.668473,0.019802,1.007055,0.061386,N/A,0.106337


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
cv_dataframe.sort_values(by = 'RV Score', inplace = True)
cv_dataframe = rv_dataframe[:50]
cv_dataframe.reset_index(drop = True, inplace = True)

NameError: name 'rv_dataframe' is not defined

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_size = 500000

In [ ]:
position_size = float(portfolio_size) / len(cv_dataframe.index)
for i in range(0, len(cv_dataframe['Ticker'])-1):
    cv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / cv_dataframe['Price'][i])
cv_dataframe

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
cv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [ ]:
writer.save()